In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [162]:
df_userid =  pd.read_csv('takehome_users.csv',encoding='latin-1')

In [163]:
df_userid.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [189]:
df_userid.org_id.unique()


array([ 11,   1,  94, 193, 197,  37,  74, 302, 318,  69, 130, 254,   0,
       175, 211,   2,  58,  54,   7,   6, 123,  81,  56, 134,  92, 251,
       283,  98, 401,  33,  59, 109, 276, 208, 332,  64, 235,  63,  14,
         4, 127,  32, 224, 304,  61,  51, 229,  88, 325, 150, 347, 203,
        21, 409, 215, 141,   5, 107, 184, 310, 367, 389,  89, 242,  68,
        16,  49, 185,  91, 172,   8,  43, 249,  26, 139,  70,  10, 252,
        53, 119,   3, 196, 335, 375, 303, 307,  25, 370,  83,  27, 306,
       118, 295,  19, 301, 100, 112, 263,  39,  12,  86, 402,  42, 103,
       373, 268,  30, 406,  48, 313,   9, 138, 261,  35, 264,  95,  55,
        60, 234,  28, 410, 170, 374,  15,  17, 364,  62, 244, 342, 225,
       121, 255, 411, 217, 165, 415, 104, 329, 278, 116, 151, 290, 200,
       187, 122, 353, 135, 328,  36,  13, 341, 349, 198, 153, 108,  77,
       142, 275, 176,  87, 191, 102, 245, 154, 218, 385,  76, 321, 159,
       371,  50,  75, 333, 137, 292, 286, 131, 178, 213, 376, 36

In [164]:
df_userid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [216]:
df_usage =pd.read_csv('takehome_user_engagement.csv',parse_dates=['time_stamp'] )
df_usage = df_usage.set_index(pd.DatetimeIndex(df_usage['time_stamp']))

In [217]:
df_usage.head()

,time_stamp,user_id,visited
time_stamp,,,
2014-04-22 03:53:30,2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2013-12-25 03:45:04,2,1


In [218]:
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 207917 entries, 2014-04-22 03:53:30 to 2014-01-26 08:57:12
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 6.3 MB


In [219]:
len(df_usage.user_id.unique())

8823

In [220]:
# Group by userid and resample the timeseries data with "1W'to get  number of times any user has logged in 'in a 7 day timeframe'
grouped = df_usage.groupby('user_id').resample('1W').sum()

In [221]:
# filter using 'loc' to find the no. of times the login was greater than '3' in the 7 day timeframe'
active_users = grouped.loc[(grouped['visited'] >=3),['visited']]
active_users.head()

visited
user_id time_stamp         
2       2014-02-09        3
10      2013-03-03        3
        2013-04-14        3
        2013-04-28        3
        2013-05-05        4

In [222]:
len(active_users)

33859

In [223]:
#Compute count within the group
active_users_agg =active_users.groupby(level='user_id').count()
active_users_agg.head()

,visited
user_id,
2,1
10,52
20,1
33,1
42,68


In [224]:
df_userid['Active'] = 0

In [225]:
# merge user details for each of the active users
#active_user_data = pd.merge(active_users_agg, df_userid, how= 'left', left_on = 'user_id', right_on ='object_id')
active_user_data = active_users_agg.merge(df_userid, left_index = True,left_on ='user_id', right_on = 'object_id', how = 'inner')
active_user_data['Active']=1
active_user_data

,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,Active
1,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
9,52,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,1
19,1,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,1
32,1,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,1
41,68,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,1
42,1,43,2012-12-31 07:13:17,Gomes Beatrice,vgqmpljq@yyyxt.com,GUEST_INVITE,1.366010e+09,0,0,63,149.0,1
52,1,53,2013-02-09 23:47:15,Ribeiro Camila,CamilaGomesRibeiro@gmail.com,GUEST_INVITE,1.367798e+09,0,0,37,3641.0,1
62,62,63,2012-12-20 16:30:52,Martins Leonor,LeonorAraujoMartins@gmail.com,SIGNUP_GOOGLE_AUTH,1.401899e+09,0,0,203,NaN,1
68,91,69,2012-06-29 10:15:18,Bowen Alisha,AlishaBowen@gustr.com,GUEST_INVITE,1.401445e+09,0,0,107,5022.0,1
73,8,74,2013-01-14 00:02:41,Farrell Jack,JackFarrell@gmail.com,SIGNUP_GOOGLE_AUTH,1.377303e+09,0,0,89,NaN,1


In [226]:
df_userid.iloc[list(active_user_data.index),10]=1

In [227]:
df_userid

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,Active
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,0
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,1


In [260]:
feature_df = df_userid.set_index('object_id').sort_index()

In [261]:
feature_df = feature_df.drop(['name','email','last_session_creation_time'],axis=1)

In [262]:
feature_df = feature_df.drop('creation_time', axis =1)

In [263]:
feature_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,Active
object_id,,,,,,
1,GUEST_INVITE,1,0,11,10803.0,0
2,ORG_INVITE,0,0,1,316.0,1
3,ORG_INVITE,0,0,94,1525.0,0
4,GUEST_INVITE,0,0,1,5151.0,0
5,GUEST_INVITE,0,0,193,5240.0,0


In [264]:
feature_df['invited_by_user_id'] = feature_df['invited_by_user_id'].fillna(0)
feature_df['invited_by_user_id'] = feature_df['invited_by_user_id'].apply(lambda x: 1 if x!=0 else 0)

In [265]:
labelEncoder = LabelEncoder()
hotEncoder = OneHotEncoder(sparse=False)

creationsource_encoder = labelEncoder.fit_transform(feature_df.creation_source)
creationsource_encoder = creationsource_encoder.reshape(len(feature_df),1)
onehot_encoder = hotEncoder.fit_transform(creationsource_encoder)
creationsource_df = pd.DataFrame(onehot_encoder,columns=labelEncoder.classes_,dtype='int64')

#org_id_encoder = labelEncoder.fit_transform(feature_df.org_id)
#org_id_encoder = org_id_encoder.reshape(len(feature_df),1)
#onehot_encoder = hotEncoder.fit_transform(org_id_encoder)
#org_id_encoder_df = pd.DataFrame(onehot_encoder, columns=labelEncoder.classes_,dtype='int64')

In [266]:
#org_id_encoder_df
org = feature_df.groupby('org_id').sum()

org_cata = []
for i in feature_df['org_id']:
    cata = org.loc[i,'Active']
    org_cata.append(cata)
feature_df['org_id'] = org_cata

feature_df = pd.get_dummies(data=feature_df,columns=['org_id'])
feature_df.head()


,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,Active,org_id_0,org_id_1,org_id_2,org_id_3,org_id_4,...,org_id_6,org_id_7,org_id_8,org_id_9,org_id_10,org_id_11,org_id_12,org_id_13,org_id_14,org_id_16
object_id,,,,,,,,,,,,,,,,,,,,,
1,GUEST_INVITE,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,ORG_INVITE,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,ORG_INVITE,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,GUEST_INVITE,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,GUEST_INVITE,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
feature_df.index = np.arange(0,len(feature_df.index))
feature_df = feature_df.join(creationsource_df)

In [268]:
feature_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,Active,org_id_0,org_id_1,org_id_2,org_id_3,org_id_4,...,org_id_11,org_id_12,org_id_13,org_id_14,org_id_16,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,GUEST_INVITE,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,ORG_INVITE,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,ORG_INVITE,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,GUEST_INVITE,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,GUEST_INVITE,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [269]:
feature_df = feature_df.drop('creation_source', axis =1)

In [270]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 25 columns):
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
invited_by_user_id            12000 non-null int64
Active                        12000 non-null int64
org_id_0                      12000 non-null uint8
org_id_1                      12000 non-null uint8
org_id_2                      12000 non-null uint8
org_id_3                      12000 non-null uint8
org_id_4                      12000 non-null uint8
org_id_5                      12000 non-null uint8
org_id_6                      12000 non-null uint8
org_id_7                      12000 non-null uint8
org_id_8                      12000 non-null uint8
org_id_9                      12000 non-null uint8
org_id_10                     12000 non-null uint8
org_id_11                     12000 non-null uint8
org_id_12                     12000 non-null uint8
org_id_13               

In [271]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV,train_test_split

y = feature_df.pop('Active')
x = feature_df

In [272]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.25,random_state=42)

In [273]:

graboost = GradientBoostingClassifier(max_features='sqrt')
parametergra = {'n_estimators':[40,45,50,55,60],'learning_rate':[.01,0.02,0.03,0.04,0.05],
              'max_depth':[1,2,3,4,5]}
grid1 = GridSearchCV(estimator=graboost,param_grid=parametergra,
                     scoring='accuracy',cv=5)
grid1.fit(x_train,y_train)
best_para_gra = grid1.best_params_
best_acc_gra = grid1.best_score_

print('The Tuned Paratmers :\n',best_para_gra,'\nAchieved %s Percent Accuracy' %(best_acc_gra*100))

The Tuned Paratmers :
 {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 40} 
Achieved 88.0 Percent Accuracy


In [243]:
gbm = GradientBoostingClassifier(max_features='sqrt',
                                 n_estimators=50,learning_rate=.05,max_depth= 3)
gbm.fit(x_train,y_train)
feature_coef = pd.DataFrame(gbm.feature_importances_).transpose()
feature_coef.columns = list(feature_df.columns)
feature_coef.index = ['GBM'] 
feature_coef.transpose().sort_values(by='GBM',ascending=False)

,GBM
PERSONAL_PROJECTS,0.156322
org_id_0,0.143398
org_id_5,0.122176
org_id_1,0.104440
org_id_11,0.099417
org_id_6,0.074986
GUEST_INVITE,0.057852
org_id_12,0.041035
SIGNUP_GOOGLE_AUTH,0.027259
org_id_7,0.023222


Using the feature_importance of GBM, we arrive at the  independent features  which  influence the dependent variable  are given in sorted order in decreasing order of importance.